In [ ]:
import numpy as np
import sys
import os
from pymatreader import read_mat
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils import data
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
from sklearn.metrics import average_precision_score
import cv2


In [ ]:
motionname_ad = []
filenames_ad = []
seq_ad = []

for filename in os.listdir(r'C:\Users\prada\Documents\Work Related\RA\ad 9 motions 5 trials'):
    filenames_ad.append('C:\\Users\prada\\Documents\\Work Related\\RA\\ad 9 motions 5 trials\\' + filename)
for filename in filenames_ad:
        data1 = read_mat(filename)
        motionname_ad.append(data1.get('motionName'))
        seq_ad.append(data1.get('seq'))

In [ ]:
motionname_ad = np.asarray(motionname_ad)
seq_ad = np.asarray(seq_ad)


In [ ]:

width = 227
height = 227
final_ad = np.zeros((250*45,width, height,3), dtype = 'uint8')
a = 0
for idx in range(45):
    for i in range(250):
        img = seq_ad[idx, :, :, i]
        img_sm = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)
        img_bgr = cv2.cvtColor(img_sm, cv2.COLOR_GRAY2BGR)
        img_bgr = np.asarray(img_bgr,dtype = 'uint8')
        final_ad[a] = img_bgr
        a = a + 1

In [ ]:
traindata = final_ad
labels = motionname_ad

In [ ]:
traindata.shape

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)


In [ ]:
labels = np.repeat(integer_encoded, repeats = 250, axis=0)

In [ ]:
labels.shape

In [ ]:
!rm -rf ~/.nv

In [ ]:
del filename,filenames_ad,final_ad,width,seq_ad,data1,img,img_sm,idx

In [ ]:
traindata.shape,labels.shape

In [ ]:
traindata = traindata.T

In [ ]:
a,b,c = traindata[:,:,:,0:2250] , traindata[:,:,:,4500:6750], traindata[:,:,:,9000:11250]

In [ ]:
X_train = np.concatenate((a,b,c),axis=3)

In [ ]:
X_val = traindata[:,:,:,2250:5000]

In [ ]:
X_test = traindata[:,:,:,6750:9000]

In [ ]:
j,k,l = labels[0:2250],labels[4500:6750],labels[9000:11250]

In [ ]:
y_train = np.concatenate((j,k,l),axis=0)

In [ ]:
y_val = labels[2250:5000]

In [ ]:
y_test = labels[6750:9000]

In [ ]:
X_train.shape

In [ ]:
X_train = np.transpose(X_train,(3,0,2,1))

In [ ]:
X_val = np.transpose(X_val,(3,0,2,1))
X_test = np.transpose(X_test,(3,0,2,1))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
del a,b,c,j,k,l,

In [ ]:
device

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

In [ ]:
X_train,y_train = torch.from_numpy(X_train).float(), torch.from_numpy(y_train.T).long()

In [ ]:
train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])



In [ ]:
my_dataset = CustomTensorDataset(tensors=(X_train, y_train), transform=train_transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=my_dataset,
                                               batch_size=48, 
                                               shuffle=True,
                                               num_workers=0)

In [ ]:
X_val,y_val = torch.from_numpy(X_val).float(), torch.from_numpy(y_val.T).long()

In [ ]:
val_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

In [ ]:
X_val.shape

In [ ]:
my_val_dataset = CustomTensorDataset(tensors=(X_val, y_val), transform=val_transform)

In [ ]:
val_loader = torch.utils.data.DataLoader(dataset=my_val_dataset,
                                               batch_size=48, 
                                               shuffle=True,
                                               num_workers=0)

In [ ]:

torch.cuda.empty_cache()

In [ ]:
import torchvision.models as models
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512, 9)

In [ ]:
classifier = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters())

In [ ]:
def train(epoch):
    print('Epoch: %d' % epoch)
    classifier.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = classifier(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print('Train Loss: %.3f | Acc: %.3f%% (%d/%d)'
          % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [ ]:
def test(epoch):
    classifier.eval()
    test_loss = 0
    pre = np.empty(0)
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = classifier(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            for i in range(0,len(predicted)):
                if predicted[i]==targets[i]:
                    pre = np.append(pre,predicted[i].cpu().numpy())
            #pre = np.append(pre,predicted.cpu().numpy())
        #scheduler.step(test_loss)
        print(' Val Loss: %.3f | Acc: %.3f%% (%d/%d)'
              % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        

In [ ]:
for epoch in range(25):
    train(epoch)
    test(epoch)
    scheduler.step()